In [1]:
from modules.statistical import *

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:02<00:00,  2.49it/s]


In [2]:
layers = list(range(1,14))
layers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [3]:
analyzer_launcher(outputs_name="outputs10", analytical_func=compare_MlC_MCA_for_GLO_PER_GAP, layers=layers, approach=approach, aggregation_f=None,logics=None, alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.85])

True
qualitative
/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/outputs10/GERMAN/0.1/qualitative/model_storage
bestK: 5, serialized objects: {'model': {5: 1.9737, 0: 1.3158, 2: 1.3158, 8: 1.3158, 3: 0.6579, 12: 0.6579, 4: 0.0, 6: 0.0, 10: 0.0, 1: -0.6579, 11: -0.6579, 7: -1.3158, 9: -1.9737}, 'nbGood': 6, 'bestK': 5, 'name': ['Attribute_9', 'Attribute_1', 'Attribute_4', 'Attribute_14', 'Attribute_6', 'Attribute_20', 'Attribute_7', 'Attribute_10', 'Attribute_17', 'Attribute_3', 'Attribute_19', 'Attribute_12', 'Attribute_15']}
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF', 'XGB']
['LDA', 'LR', 'SVM', 'DT', 'RF',

IndexError: list index out of range

In [ ]:
[
    analyzer_launcher_for_descriptor_rank_plot(
        outputs_name="outputs10", 
        analytical_func=Descriptors_comparison, 
        approach=['mixed'],
        layers=layers, 
        aggregation_f=statistics.mean, 
        top=k, 
        alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
        with_class=True
    )  for k in [10,15,20,15,25,30]
]